## Import libraries

In [1]:
import os
import re
import yaml
import json
import pandas as pd
import numpy as np
from pybis import Openbis
import warnings
import string
warnings.filterwarnings("ignore")

## Functions

In [21]:
def load_json(filepath: str) -> dict:
    return json.load(open(filepath, "r"))

## Classes

In [39]:
class OpenBisDatabase:
    def __init__(self, openbis_url: str, openbis_user: str):
        self.openbis_url = openbis_url
        self.openbis_user = openbis_user
    
    def log_in(self, openbis_pw: str):
        """Function to login to openBIS."""
        if Openbis(self.openbis_url, verify_certificates=False).is_token_valid():
            self.session = Openbis(self.openbis_url, verify_certificates=False)
        else:
            Openbis(self.openbis_url, verify_certificates=False).login(self.openbis_user, openbis_pw, save_token=True)
            self.session = Openbis(self.openbis_url, verify_certificates=False)
    
    def load_yaml(self, filepath: str):
        self.objects_schema = yaml.safe_load(open(filepath, 'r'))
    
    def show_object_in_openbis_menu(self, object_type_code: str):
        settings_sample = self.session.get_sample("/ELN_SETTINGS/GENERAL_ELN_SETTINGS")
        try:
            settings = json.loads(settings_sample.props["$eln_settings"])
            settings['sampleTypeDefinitionsExtension'][object_type_code] = {'SAMPLE_PARENTS_DISABLED': False,
                                                                            'SAMPLE_PARENTS_ANY_TYPE_DISABLED': False,
                                                                            'SAMPLE_CHILDREN_DISABLED': False,
                                                                            'SAMPLE_CHILDREN_ANY_TYPE_DISABLED': False,
                                                                            'USE_AS_PROTOCOL': False,
                                                                            'ENABLE_STORAGE': False,
                                                                            'SHOW': True, 
                                                                            'SHOW_ON_NAV': True}
            settings_sample.props['$eln_settings'] = json.dumps(settings)
            settings_sample.save()
        except TypeError:
            print("The object is not available in the main menu. For that, open openBIS, go to Settings, click on Edit, open one object type in the Object Type definitions Extension and add to the main menu. Then, this function works.")
    
    def create_object_type(self, object_type_details: dict):
        object_type_dict = {
            "code": object_type_details["annotations"]["openbis_label"].replace(" ", "_"),
            "generatedCodePrefix": object_type_details["annotations"]["openbis_code"]
        }

        try:
            object_type = self.session.get_object_type(object_type_dict["code"])
        except ValueError:
            print(f"{object_type_dict['code']} does not exist.")
            object_type = self.session.new_object_type(
                autoGeneratedCode=True, 
                subcodeUnique=False,
                listable=True,
                showContainer=False,
                showParents=True,
                showParentMetadata=False,
                **object_type_dict
            )
            object_type.save()
        
        return object_type

    def assign_properties_to_object_type(self, object_type: str, object_property_types: list[str]):
        for property_code in object_property_types:
            try:
                object_type.assign_property(self.session.get_property_type(code=property_code))
            except ValueError:
                print(f"{property_code} does not exist.")
    
    def create_vocabulary(self, property_type: str) -> str:
        property_type_range = self.objects_schema["slots"][property_type]["range"]
        vocabularies = self.objects_schema["enums"]

        vocabulary_dict = {
            "code": property_type_range,
            "description": vocabularies[property_type_range]["description"],
            "terms": []
        }
        
        for term, term_details in vocabularies[property_type_range]["permissible_values"].items():
            term_dict = {
                "code": term,
                "description": term_details["description"],
                "label": term_details["annotations"]["openbis_label"]
            }
            
            vocabulary_dict["terms"].append(term_dict)
        
        try:
            self.session.new_vocabulary(**vocabulary_dict).save()
        except ValueError:
            print(f"{vocabulary_dict['code']} already exists.")
        
        return vocabulary_dict["code"]
    
    def generate_property_type_dictionary(self, property_type: str) -> dict:
        property_type_dict = {
            "code": property_type,
            "label": self.objects_schema["slots"][property_type]["annotations"]["openbis_label"],
            "description": self.objects_schema["slots"][property_type]["description"],
            "dataType": self.objects_schema["slots"][property_type]["annotations"]["openbis_type"],
            "managedInternally": False
        }
        
        if property_type_dict["dataType"] == "MULTILINE_VARCHAR":
            property_type_dict["metaData"] = {"custom_widget": "Word Processor"}
        
        if property_type_dict["dataType"] == "CONTROLLEDVOCABULARY":
            property_type_dict["vocabulary"] = self.create_vocabulary(property_type)
        
        return property_type_dict
    
    def create_property_type(self, property_type: str) -> str:
        if property_type == "name":
            return "$name"
        
        if property_type == "description":
            return "description"
        
        if self.objects_schema["slots"][property_type]["annotations"]["openbis_type"] not in ["Not used", "OBJECT"]:
            
            property_type_dict = self.generate_property_type_dictionary(property_type)
            
            try:
                self.session.new_property_type(**property_type_dict).save()
            except ValueError:
                print(f"{property_type_dict['code']} already exists.")
            
            return property_type
        
        else:
            return None
    
    def setup_openbis_database(self):
        # Create object types in openBIS following the LinkML schema
        for key, object in self.objects_schema["classes"].items():
            object_property_types = []
            if object["annotations"]["openbis_object"]:
                if "slots" in object:
                    for property_type in object["slots"]:
                        property_type = self.create_property_type(property_type)
                        if property_type is not None:
                            object_property_types.append(property_type)

                    object_type = self.create_object_type(object)
                    self.assign_properties_to_object_type(object_type, object_property_types)
                    self.show_object_in_openbis_menu(object_type.code)
        
        # Create experiment type EXPERIMENT
        try:
            experiment_type = self.session.new_experiment_type("EXPERIMENT", description = "Experiment")
            experiment_type.save()
            experiment_type.assign_property(prop = '$name', section = 'General info')
        except ValueError:
            print(f"Experiment type EXPERIMENT exists already.")
        
        # Create collections in openBIS
        collections_info = load_json("collections_info.json")
        
        for space_code, space_info in collections_info["spaces"].items():
            try:
                space = self.session.new_space(code=space_code, description = space_info["description"]).save()
            except ValueError:
                print(f"Space {space_code} exists already.")
                space = self.session.get_space(code = space_code)
            if "projects" in space_info:
                for project_code, project_info in space_info["projects"].items():
                    try:
                        space.new_project(code=project_code, description = project_info["description"]).save()
                    except ValueError:
                        print(f"Project {project_code} exists already.")
                        project = self.session.get_projects(space = space_code, code = project_code)[0]
                    if "experiments" in project_info:
                        for experiment_code, experiment_info in project_info["experiments"].items():
                            try:
                                experiment = self.session.new_experiment(code = experiment_code, type = "COLLECTION", project = f"/{space_code}/{project_code}/")
                                experiment.set_props({"$name": experiment_info["name"]})
                                experiment.save()
                            except ValueError:
                                print(f"Experiment {experiment_code} exists already.")

## Create object types in openBIS

In [40]:
# Create openBIS database instance
openbis_database = OpenBisDatabase(
    openbis_url = 'openbis', 
    openbis_user = 'admin'
)

# Connect to openBIS
openbis_database.log_in(openbis_pw = '123456789')

# Load LinkML schema
yaml_filepath = os.path.join(os.sep, "home", "jovyan", "aiida-openbis", "Notebooks", "Metadata_Schemas_LinkML", "materialMLinfo.yaml")
openbis_database.load_yaml(yaml_filepath)

# Setup openBIS database (create object types, experiment type, and collections)
openbis_database.setup_openbis_database()

#TODO: Document the classes and functions and try to simplify

{"method": "createExperimentTypes", "params": ["admin-240715140416454x12C13E6E590174D9274F832ECBBE45CB", [{"@type": "as.dto.experiment.create.ExperimentTypeCreation", "code": "EXPERIMENT", "description": "Experiment", "validationPluginId": null}]], "id": "2", "jsonrpc": "2.0"}
Experiment type EXPERIMENT exists already.
{"method": "createSpaces", "params": ["admin-240715140416454x12C13E6E590174D9274F832ECBBE45CB", [{"@type": "as.dto.space.create.SpaceCreation", "code": "CARBON_NANOMATERIALS", "description": "Carbon Nanomaterials"}]], "id": "2", "jsonrpc": "2.0"}
Space CARBON_NANOMATERIALS exists already.
{"method": "createSpaces", "params": ["admin-240715140416454x12C13E6E590174D9274F832ECBBE45CB", [{"@type": "as.dto.space.create.SpaceCreation", "code": "MATERIALS_TO_DEVICES", "description": "Materials to Devices"}]], "id": "2", "jsonrpc": "2.0"}
Space MATERIALS_TO_DEVICES exists already.
{"method": "createSpaces", "params": ["admin-240715140416454x12C13E6E590174D9274F832ECBBE45CB", [{"